# 0. 기본 정보
* 코드 작성자: 여서연
* 코드 작성일: 2025-04-17 ~
* 코드 작성 목적: 코사인 유사도 기반 추천 시스템 구현

# 1. 기초 설정

## 사용 라이브러리

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
import warnings
warnings.filterwarnings('ignore')

## 데이터 불러오기

In [4]:
crawled = pd.read_csv('../data/dbpia_crawling_result.csv')

In [5]:
df_cleaned_komoran = pd.read_csv("../data/cleaned_docs_komoran.csv", encoding='utf-8-sig')
df_cleaned_komoran['cleaned_doc'] = df_cleaned_komoran['cleaned_doc'].fillna('')
df_cleaned_komoran.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 798 entries, 0 to 797
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   cleaned_doc  798 non-null    object
dtypes: object(1)
memory usage: 6.4+ KB


In [6]:
df_cleaned_okt = pd.read_csv("../data/cleaned_docs_okt.csv", encoding='utf-8-sig')
df_cleaned_okt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 798 entries, 0 to 797
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   cleaned_doc  798 non-null    object
dtypes: object(1)
memory usage: 6.4+ KB


In [7]:
terms_komoran = [doc.split() for doc in df_cleaned_komoran['cleaned_doc']]
print(terms_komoran[:5])

[['어장', '관리', '철학', '고찰', '최근', '한국', '사회', '유통', '시작', '신조어', '연인', '관계', '발전', '이전', '단계', '남녀', '관계', '뜻', '이해', '의미', '활동', '정체', '여전', '오리무중', '사람', '사람', '답변', '배경', '하', '필자', '미국', '철학자', '해리', '프랑크푸르트', '인간관', '의거', '의지', '불확정성', '개념', '도입', '개념', '본성', '포착', '필자', '제안', '상대방', '성적', '호감', '남녀', '만남', '시작', '자신', '의지', '불확정성', '과정', '핵심', '필자', '제안', '어장', '관리', '관계', '흥미', '시각', '제공', '논문', '말미', '논의'], ['인공', '지능', '인공', '감정', '감정', '기계', '실현', '인공', '감정', '철학', '탐구', '필요', '시점', '인간', '고유', '영역', '간주', '인지', '과제', '기계', '추월', '염려', '처지', '사람', '이성', '감정', '인간', '고유', '최근', '인공지능', '로봇', '감성', '작업', '화두', '글', '인공', '감정', '실현', '잠재', '위험', '논의', '감성', '로봇', '개발', '현황', '동기', '개괄', '로봇', '감정', '감정', '소유', '로봇', '검토', '감정', '선험', '정의', '감정', '수행', '핵심', '역할', '소개', '대상', '감정', '부여', '기준', '제안', '기준', '감정', '로봇', '미래', '실현', '주장', '감정', '소유', '로봇', '등장', '이전', '정도', '자율', '로봇', '일방', '감정', '소통', '잠재', '위험', '대비', '주장'], ['논문', '철회', '관음', '충', '발생학', '한국', '남성성', '완전', '변태', '과

In [8]:
terms_okt = [doc.split() for doc in df_cleaned_okt['cleaned_doc']]
print(terms_okt[:5])

[['썸', '어장', '관리', '철학', '고찰', '최근', '한국', '사회', '유통', '시작', '신조어', '썸', '본격', '연인', '관계', '발전', '이전', '단계', '남녀', '관계', '뜻', '이해', '의미', '썸', '활동', '정체', '여전', '오리무중', '열', '사람', '열', '사람', '모두', '답변', '배경', '하', '필자', '미국', '철학자', '해리', '프랑크푸르트', '의', '인간', '관', '의거', '의지', '불확정성', '개념', '도입', '개념', '썸', '본성', '포착', '필자', '제안', '상대방', '성적', '호감', '남녀', '만남', '시작', '자신', '의지', '불확정성', '대하', '서로', '과정', '썸', '핵심', '썸', '필자', '제안', '썸', '어장', '관리', '관계', '시각', '제공', '대해', '논문', '말미', '상세', '논의'], ['인공', '지능', '인공', '감정', '감정', '기계', '실현', '인공', '감정', '관', '철학', '탐구', '시점', '인간', '고유', '영역', '간주', '인지', '과제', '기계', '추월', '염려', '처지', '사람', '이제', '이성', '감정', '인간', '유성', '최근', '인공', '지능', '로봇', '감성', '불어', '작업', '화두', '글', '인공', '감정', '실현', '가능성', '잠재', '위험', '논의', '먼저', '감성', '로봇', '개발', '현황', '주요한', '동기', '개괄', '왜', '로봇', '감정', '감정', '소유', '로봇', '검토', '감정', '선', '정의', '감정', '수행', '핵심', '역할', '소개', '이로', '대상', '감정', '부여', '기준', '제안', '기준', '감정', '로봇', '미래', '실현', '가능성', '주장', '감정', '소유', '

# 2. TF-IDF 행렬 계산

In [9]:
# 토큰 리스트를 문자열로 결합 (sklearn은 문자열 입력을 기대하므로)
docs_joined_komoran = [' '.join(doc) for doc in terms_komoran]
docs_joined_okt = [' '.join(doc) for doc in terms_okt]

In [10]:
# TF-IDF 벡터화
vectorizer = TfidfVectorizer()

tfidf_matrix_komoran = vectorizer.fit_transform(docs_joined_komoran)
tfidf_matrix_okt = vectorizer.fit_transform(docs_joined_okt)

# 3. 추천 시스템 구현

In [11]:
# 코사인 유사도 계산
cos_sim_matrix_komoran = cosine_similarity(tfidf_matrix_komoran, tfidf_matrix_komoran)
cos_sim_matrix_okt = cosine_similarity(tfidf_matrix_okt, tfidf_matrix_okt)

print('코사인 유사도 연산 결과(komoran) :', cos_sim_matrix_komoran.shape)
print('코사인 유사도 연산 결과(okt) :', cos_sim_matrix_okt.shape)

코사인 유사도 연산 결과(komoran) : (798, 798)
코사인 유사도 연산 결과(okt) : (798, 798)


In [12]:
title_to_index = dict(zip(crawled['제목'], crawled.index))
print(title_to_index)

{'썸타기와 어장관리에 대한 철학적 고찰': 0, '인공 지능에서 인공 감정으로 : 감정을 가진 기계는 실현가능한가?': 1, "[논문철회] '관음충'의 발생학: 한국남성성의 불완전변태과정(homomorphism)의 추이에 대한 신물질주의적 분석": 2, '한국의 초저출산: 무엇이 원인이고 무엇이 해법인가': 3, 'GMO의 윤리적 문제': 4, '왜 한국 남성은 한국여성들에게 분노하는가 : 여성혐오, 한국사회가 가지고 있는 어떤 특수성': 5, '안락사 사례로 보는 생명과 권리의 문제': 6, '인터넷 밈의 언어적 성격 고찰': 7, '대학 신입생 글쓰기에 나타난 문장 오류 양상 분석': 8, '4차 산업 혁명 시대, 대학 교육과 콘텐츠': 9, '배아복제 기술의 윤리적 문제와 줄기세포 연구의 한계': 10, '동물실험과 심의': 11, "강남역 살인사건부터 '메갈리아' 논쟁까지 : '페미니즘 봉기'와 한국 남성성의 위기": 12, '인공지능은 인간의 일자리를 얼마나 대체할 것인가 : 인공지능 시대의 기술과 노동에 관한 시론': 13, '썸을 탄다는 것은 무엇인가?: 신조어 “썸타다”의 적용조건 분석': 14, '니체와 프롬의 철학으로 되짚어 본 EXO의 〈으르렁〉': 15, '방탄소년단: 새로운 세대의 새로운 소통 방식, 그리고 감정노동': 16, 'GMO(유전자 조작 식품)를 어떻게 볼 것인가?': 17, '웹소설 장에서 사용되는 장르 연관 개념 연구': 18, '당의 입장에서 본 신라의 통일': 19, '동성애에 관한 핵심 쟁점 : - 범죄인가, 질병인가, 소수의 성지향인가?': 20, '숏폼 동영상 콘텐츠의 유형 연구': 21, '뇌사판정과 장기이식의 윤리적 문제': 22, '안락사와 존엄사, 그리고 웰다잉법': 23, '흥선대원군의 개혁정치와 그 한계성': 24, '한국 아이돌 콘텐츠의 트랜스미디어 스토리텔링 연구 : EXO와 BTS를 중심으로': 25, '현상학과 질적연구방법': 26, '메타버스 개념과 유형에 관한 시론 : 가능세계 이론을 중심으

In [13]:
def recommend_papers_by_title(title, cos_sim_matrix, top_k=10):
    idx = title_to_index[title]

    sim_scores = list(enumerate(cos_sim_matrix[idx]))
    sim_scores = [(i, score) for i, score in sim_scores if i != idx] # 자기 자신 제외
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[:top_k] # 유사도 순 상위 k

    recommendations = []
    for rank, (paper_idx, score) in enumerate(sim_scores, start=1):
        paper_category = crawled['카테고리'].iloc[paper_idx]
        paper_title = crawled['제목'].iloc[paper_idx]
        paper_abstract = crawled['초록'].iloc[paper_idx]
        print(f" - 추천 {rank}위: {paper_category} - [{paper_title}], (유사도 {score:.4f})")
        recommendations.append({
            '순위': rank,
            '카테고리': paper_category,
            '제목': paper_title,
            '초록': paper_abstract,
            '유사도': round(score, 4)
        })

    # 정량적 지표 - 평균 유사도 사용
    mean_sim = np.mean([score for _, score in sim_scores])
    print(f"\n ▶ 평균 유사도 (Mean Similarity): {mean_sim:.4f}")

    # DataFrame 생성 및 저장
    result_df = pd.DataFrame(recommendations)
    return result_df

# 4. 평가

In [14]:
TOP_N = 10
keyword = "코로나"
category = "사회과학"

In [15]:
filtered = crawled[((crawled['제목'] + " " + crawled['초록']).str.contains(keyword, na=False))
                   & (crawled['카테고리']==category)]
filtered.reset_index(drop=True, inplace=True)
filtered

,카테고리,제목,초록
0,사회과학,코로나19가 바꾼 소비 트렌드,등록된 정보가 없습니다.
1,사회과학,"코로나19, 언택트 사회를 가속화하다",등록된 정보가 없습니다.
2,사회과학,"코로나19 세대, 정신건강 안녕한가!",등록된 정보가 없습니다.
3,사회과학,코로나19로 인한 아동일상 변화와 정서 상태,본 연구는 코로나19로 인한 아동의 일상 변화와 정서 상태 간의 관계를 탐색하고 어...
4,사회과학,"컨택트에서 언택트로 : 마케팅, 코로나를 이겨라!",등록된 정보가 없습니다.
5,사회과학,코로나를 통해 드러난 교육의 과제와 전망,등록된 정보가 없습니다.
6,사회과학,코로나19 사태의 SNS 마케팅 강화,등록된 정보가 없습니다.
7,사회과학,"포스트 코로나19, 뉴노멀 시대의 산업 전략",등록된 정보가 없습니다.
8,사회과학,"기본소득제 - 정의, 쟁점, 전망",○ 보편적 기본소득제는 모든 국민 각자에게 아무 조건 없이 일정한 현금을 정기적으로...
9,사회과학,"코로나19가 앞당긴 미래, 교육하는 시대에서 학습하는 시대로",등록된 정보가 없습니다.


In [16]:
# komoran
print(f"기준 문헌: [{filtered['제목'][0]}]")
result_komoran = recommend_papers_by_title(filtered['제목'][0], cos_sim_matrix_komoran, TOP_N)

기준 문헌: [코로나19가 바꾼 소비 트렌드]
 - 추천 1위: 복합학 - [추천 시스템에 의한 필터 버블 개선 방향 제시 : 유투브를 통해서], (유사도 0.5962)
 - 추천 2위: 사회과학 - [플렉스 소비 트렌드], (유사도 0.4990)
 - 추천 3위: 사회과학 - [코로나19, 언택트 사회를 가속화하다], (유사도 0.4671)
 - 추천 4위: 복합학 - [줄기세포에 대한 윤리적ㆍ법적 논쟁의 변화], (유사도 0.4437)
 - 추천 5위: 인문학 - [코로나19 백신 연구와 개발], (유사도 0.4282)
 - 추천 6위: 사회과학 - [코로나19 사태의 SNS 마케팅 강화], (유사도 0.3863)
 - 추천 7위: 의약학 - [교대근무 간호사의 직무스트레스, 수면의 질, 피로가 직무몰입에 미치는 영향], (유사도 0.3673)
 - 추천 8위: 의약학 - [암 환자를 돌보는 종합병원 간호사의 공감역량, 의사소통능력, 간호근무환경이 인간중심돌봄에 미치는 영향], (유사도 0.3508)
 - 추천 9위: 예술체육학 - [노인을 위한 웨어러블 헬스케어 디바이스 개발 동향 연구], (유사도 0.3489)
 - 추천 10위: 사회과학 - [코로나19 세대, 정신건강 안녕한가!], (유사도 0.3450)

 ▶ 평균 유사도 (Mean Similarity): 0.4233


In [17]:
filtered_komoran = result_komoran[((result_komoran['제목'] + " " + result_komoran['초록']).str.contains(keyword, na=False))
                                  & (result_komoran['카테고리'] == category)]
filtered_komoran

,순위,카테고리,제목,초록,유사도
2,3,사회과학,"코로나19, 언택트 사회를 가속화하다",등록된 정보가 없습니다.,0.4671
5,6,사회과학,코로나19 사태의 SNS 마케팅 강화,등록된 정보가 없습니다.,0.3863
9,10,사회과학,"코로나19 세대, 정신건강 안녕한가!",등록된 정보가 없습니다.,0.3450


In [18]:
precision_komoran = len(filtered_komoran)/TOP_N
recall_komoran = len(filtered_komoran)/(len(filtered) - 1)
fscore_komoran = 2*precision_komoran*recall_komoran/(precision_komoran+recall_komoran)

print(f'Precision: {precision_komoran}\nRecall: {recall_komoran}\nF-Score: {fscore_komoran}')

Precision: 0.3
Recall: 0.21428571428571427
F-Score: 0.25


In [19]:
# okt
print(f"기준 문헌: [{filtered['제목'][0]}]")
result_okt = recommend_papers_by_title(filtered['제목'][0], cos_sim_matrix_okt, TOP_N)

기준 문헌: [코로나19가 바꾼 소비 트렌드]
 - 추천 1위: 사회과학 - [플렉스 소비 트렌드], (유사도 0.5706)
 - 추천 2위: 복합학 - [줄기세포에 대한 윤리적ㆍ법적 논쟁의 변화], (유사도 0.5346)
 - 추천 3위: 복합학 - [추천 시스템에 의한 필터 버블 개선 방향 제시 : 유투브를 통해서], (유사도 0.4229)
 - 추천 4위: 농수해양학 - [작두콩 추출물의 화학적 특성 및 DPPH 라디컬 소거능], (유사도 0.3037)
 - 추천 5위: 예술체육학 - [코로나19 이후, 여가활동의 변화 양상], (유사도 0.2897)
 - 추천 6위: 복합학 - [엔터테인먼트 영역에서의 메타버스 인식과 수용방식 연구 : 에스파(aespa) 팬덤 사례를 중심으로], (유사도 0.2794)
 - 추천 7위: 사회과학 - [코로나를 통해 드러난 교육의 과제와 전망], (유사도 0.2639)
 - 추천 8위: 인문학 - [코로나19 백신 연구와 개발], (유사도 0.2633)
 - 추천 9위: 사회과학 - [코로나19 세대, 정신건강 안녕한가!], (유사도 0.2606)
 - 추천 10위: 사회과학 - [1인 가구와 방송 트렌드 변화 : 먹방, 쿡방을 중심으로], (유사도 0.2471)

 ▶ 평균 유사도 (Mean Similarity): 0.3436


In [20]:
filtered_okt = result_okt[((result_okt['제목'] + " " + result_okt['초록']).str.contains(keyword, na=False))
                                  & (result_okt['카테고리'] == category)]
filtered_okt

,순위,카테고리,제목,초록,유사도
6,7,사회과학,코로나를 통해 드러난 교육의 과제와 전망,등록된 정보가 없습니다.,0.2639
8,9,사회과학,"코로나19 세대, 정신건강 안녕한가!",등록된 정보가 없습니다.,0.2606


In [21]:
precision_okt = len(filtered_okt)/TOP_N
recall_okt = len(filtered_okt)/(len(filtered) - 1)
fscore_okt = 2*precision_okt*recall_okt/(precision_okt+recall_okt)

print(f'Precision: {precision_okt}\nRecall: {recall_okt}\nF-Score: {fscore_okt}')

Precision: 0.2
Recall: 0.14285714285714285
F-Score: 0.16666666666666666
